In [10]:
# | default_exp tools.registry

In [8]:
# | export
from typing import Dict, List, Any, Optional
from agentic.tools.base import BaseTool, ToolCategory


In [9]:
#| export
class ToolRegistry:
    """Registry for managing tools"""
    
    def __init__(self):
        self.tools: Dict[str, BaseTool] = {}
        self.categories: Dict[ToolCategory, List[str]] = {
            category: [] for category in ToolCategory
        }
    
    def register_tool(self, tool: BaseTool):
        """Register a tool"""
        if tool.name in self.tools:
            raise ValueError(f"BaseTool '{tool.name}' is already registered")
        
        self.tools[tool.name] = tool
        self.categories[tool.category].append(tool.name)
    
    def unregister_tool(self, tool_name: str):
        """Unregister a tool"""
        if tool_name not in self.tools:
            raise ValueError(f"BaseTool '{tool_name}' is not registered")
        
        tool = self.tools[tool_name]
        self.categories[tool.category].remove(tool_name)
        del self.tools[tool_name]
    
    def get_tool(self, tool_name: str) -> Optional[BaseTool]:
        """Get a tool by name"""
        return self.tools.get(tool_name)
    
    def list_tools(self, category: Optional[ToolCategory] = None) -> List[str]:
        """List all tools or tools in a specific category"""
        if category:
            return self.categories.get(category, [])
        return list(self.tools.keys())
    
    def get_tools_by_category(self) -> Dict[str, List[str]]:
        """Get tools organized by category"""
        return {
            category.value: tool_names 
            for category, tool_names in self.categories.items()
            if tool_names
        }
    
    def execute_tool(self, tool_name: str, parameters: Dict[str, Any]) -> Dict[str, Any]:
        """Execute a tool"""
        tool = self.get_tool(tool_name)
        if not tool:
            return {"error": f"BaseTool '{tool_name}' not found"}
        
        try:
            # Validate parameters
            if not tool.validate_parameters(parameters):
                return {"error": f"Invalid parameters for tool '{tool_name}'"}
            
            # Execute tool
            result = tool.execute(**parameters)
            
            # Ensure result is a dictionary
            if not isinstance(result, dict):
                result = {"result": result}
            
            return result
            
        except Exception as e:
            return {"error": f"BaseTool execution failed: {str(e)}"}
    
    def get_openai_schemas(self, tool_names: Optional[List[str]] = None) -> List[Dict[str, Any]]:
        """Get OpenAI-compatible schemas for tools"""
        if tool_names is None:
            tool_names = list(self.tools.keys())
        
        schemas = []
        for tool_name in tool_names:
            tool = self.get_tool(tool_name)
            if tool:
                schemas.append(tool.get_openai_schema())
        
        return schemas
    
    def get_tool_info(self, tool_name: str) -> Optional[Dict[str, Any]]:
        """Get detailed information about a tool"""
        tool = self.get_tool(tool_name)
        return tool.get_info() if tool else None
    
    def search_tools(self, query: str) -> List[str]:
        """Search tools by name or description"""
        query_lower = query.lower()
        matching_tools = []
        
        for tool_name, tool in self.tools.items():
            if (query_lower in tool_name.lower() or 
                query_lower in tool.description.lower()):
                matching_tools.append(tool_name)
        
        return matching_tools


In [12]:
# Example usage 
tr = ToolRegistry()
from agentic.tools.fs_read import FsReadTool
from agentic.tools.fs_write import FsWriteTool
tr.register_tool(FsReadTool())

AttributeError: 'FsReadTool' object has no attribute 'name'

In [14]:
FsWriteTool().name

'fs_write'

In [15]:
FsReadTool().name

AttributeError: 'FsReadTool' object has no attribute 'name'